In [1]:
import msoffcrypto
import io
import pandas as pd
from pyxlsb import open_workbook
import os
import numpy as np
import calendar

C:\Users\Neil\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
folder_path = "C:/Users/Neil/OneDrive/Documents/Capstone Files/CAR1_4"
password = "CRQAD_amf_2025!"

# List all .xlsb files in the folder
xlsb_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsb")]

dfs = {}  # Dictionary to store DataFrames

for file in xlsb_files:
    file_path = os.path.join(folder_path, file)

    decrypted = io.BytesIO()
    with open(file_path, "rb") as f:
        office_file = msoffcrypto.OfficeFile(f)
        office_file.load_key(password=password)
        office_file.decrypt(decrypted)

    # Read decrypted .xlsb file
    df = []
    with open_workbook(decrypted) as wb:
        with wb.get_sheet(1) as sheet:  # Read first sheet
            for row in sheet.rows():
                df.append([item.v for item in row])

    df = pd.DataFrame(df[1:], columns=df[0])  # Convert to pandas DataFrame
    dfs[file] = df  # Store DataFrame with file name as key

    print(f"Loaded: {file}")

Loaded: 01 ILS CAR JAN 2015.xlsb
Loaded: 02 ILS CAR FEB 2015.xlsb
Loaded: 03 ILS CAR MAR 2015.xlsb
Loaded: 04 ILS CAR APR 2015.xlsb
Loaded: 05 ILS CAR MAY 2015.xlsb
Loaded: 06 ILS CAR JUN 2015.xlsb
Loaded: 07 ILS CAR JUL 2015.xlsb
Loaded: 08 ILS CAR AUG 2015.xlsb
Loaded: 09 ILS CAR SEP 2015.xlsb
Loaded: 10 ILS CAR OCT 2015.xlsb
Loaded: 11 ILS CAR NOV 2015.xlsb
Loaded: 12 ILS CAR DEC 2015.xlsb
Loaded: 13 ILS CAR JAN 2016.xlsb
Loaded: 14 ILS CAR FEB 2016.xlsb
Loaded: 15 ILS CAR MAR 2016.xlsb
Loaded: 16 ILS CAR APR 2016.xlsb
Loaded: 17 ILS CAR MAY 2016.xlsb
Loaded: 18 ILS CAR JUN 2016.xlsb
Loaded: 19 ILS CAR JUL 2016.xlsb
Loaded: 20 ILS CAR AUG 2016.xlsb
Loaded: 21 ILS CAR SEP 2016.xlsb
Loaded: 22 ILS CAR OCT 2016.xlsb
Loaded: 23 ILS CAR NOV 2016.xlsb
Loaded: 24 ILS CAR DEC 2016.xlsb
Loaded: 25 ILS CAR JAN 2017.xlsb
Loaded: 26 ILS CAR FEB 2017.xlsb
Loaded: 27 ILS CAR MAR 2017.xlsb
Loaded: 28 ILS CAR APR 2017.xlsb
Loaded: 29 ILS CAR MAY 2017.xlsb
Loaded: 30 ILS CAR JUN 2017.xlsb
Loaded: 31

In [3]:
del_columns = ["DEL7", "DEL6", "DEL5", "DEL4", "DEL3", "DEL2", "DEL1"]

dfs_worst_delinquency = {}

for file, df in dfs.items():
    existing_del_cols = [col for col in del_columns if col in df.columns]

    # Compute worst delinquency level correctly
    worst_del = df[existing_del_cols].ne(0).values.argmax(axis=1)

    # Fix issue: If all values are zero, set worst_del to 0
    all_zero_mask = df[existing_del_cols].sum(axis=1) == 0  # Identify all-zero rows
    df["Worst_Del"] = 7 - worst_del
    df.loc[all_zero_mask, "Worst_Del"] = 0  # Assign 0 to rows with no delinquency

    dfs_worst_delinquency[file] = df

In [4]:
file_name = '01 ILS CAR JAN 2015.xlsb'  # Replace with the actual file name
dfs_worst_delinquency[file_name][["DEL1", "DEL2", "DEL3", "DEL4", "DEL5", "DEL6", "DEL7", "Worst_Del"]].head(30)

,DEL1,DEL2,DEL3,DEL4,DEL5,DEL6,DEL7,Worst_Del
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,14.0,13.0,10.0,0.0,0.0,0.0,0.0,3
7,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [5]:
columns_to_rename = ["Worst_Del", "DEL1"]  # Include Worst_Del and DEL1

dfs_renamed = {}

for file, df in dfs_worst_delinquency.items():
    # Extract the month and year (always at the end before .xlsb)
    filename = file.replace(".xlsb", "")  # Remove extension
    parts = filename.split()  # Split by spaces
    month_year = " ".join(parts[-2:])  # Get last two parts as "JAN 2015"

    # Rename columns
    rename_dict = {col: f"{col} {month_year}" for col in columns_to_rename if col in df.columns}
    
    df = df.rename(columns=rename_dict)
    
    # Store modified DataFrame
    dfs_renamed[file] = df

# Example: Check a renamed DataFrame
print(dfs_renamed["01 ILS CAR JAN 2015.xlsb"].head())

  APPRAISAL_DATE  APPRAISAL_RATE  APPRAISAL_VALUE   BOOK_DATE  BOOK_VALUE  \
0           None             0.0              0.0  03/21/2002         0.0   
1           None             0.0              0.0  03/21/2002         0.0   
2           None             0.0              0.0  10/25/2002         0.0   
3           None             0.0              0.0  02/19/2003         0.0   
4           None             0.0              0.0  05/15/2003         0.0   

                 CAR_DESCRIPTION  CAR_MANUFACTURER CAR_MODEL  CLOSE_DATE  \
0         2002 CHEVROLET VENTURE              10.0    2002.0  06/21/2004   
1          2002 TOYOTA ALTIS 1.6               9.0    2002.0  06/21/2005   
2  2002 ISUZU HILANDER WAGON XUV              12.0    2002.0  09/15/2006   
3          2003 ISUSU TROOPER AT              12.0    2003.0  01/10/2006   
4         2003 TOYOTA COROLLA MT               9.0    2003.0  03/20/2007   

   COLLATERAL_CONDITION  ...  NTC_AMT    RM_NO CLNT_TYPE  \
0                   

In [6]:
print(dfs_renamed["01 ILS CAR JAN 2015.xlsb"].columns)

Index(['APPRAISAL_DATE', 'APPRAISAL_RATE', 'APPRAISAL_VALUE', 'BOOK_DATE',
       'BOOK_VALUE', 'CAR_DESCRIPTION', 'CAR_MANUFACTURER', 'CAR_MODEL',
       'CLOSE_DATE', 'COLLATERAL_CONDITION', 'COLLATERAL_TYPE',
       'CREDIT_RATING', 'PDO_MAT_DATE', 'DEL1 JAN 2015', 'DEL2', 'DEL3',
       'DEL4', 'DEL5', 'DEL6', 'DEL7', 'DELQ_AGE', 'DELQ_AGE_ACTUAL',
       'DOWNPAYMENT', 'EXPIRED', 'FACE_AMT', 'FINANCED_AMT', 'GL_STATUS',
       'GL_STATUS_DESCRIPTION', 'GROSS_MONTHLY_AMORTISATION', 'ILS_ACCT_NUM',
       'LAND_VALUE', 'LAST_PAID_AMT', 'LAST_PAID_DATE', 'LOAN_DUE', 'MATURED',
       'MATURITY', 'MATURITY_DUE_CHANGE', 'NMA', 'ORIG_BOOK_DATE',
       'ORIGINAL_RATE', 'OUTSTANDING_BAL', 'PREPAYMENT', 'PRODUCT_TYPE',
       'PROPERTY_TYPE', 'LOAN_TERM', 'WORST_DELINQUENCY', 'COLLATERAL_REFNUM',
       'PURPOSE_CODE', 'PURPOSE_DESC', 'REMEDIAL_STATUS_CODE',
       'PAST_DUE_STATUS', 'DEFAULT_TAGGING', 'EXCHANGE_RATE', 'SECURITY_TYPE',
       'RE_FLOOR_AREA', 'AVAILMENT', 'CAR_TYPE', 'LOA

In [7]:
key_column = "ILS_ACCT_NUM"

# List to store DataFrames with selected columns
dfs_lst = []

for file, df in dfs_renamed.items():
    # Identify columns that start with 'DEL1' and 'Worst_Del'
    del1_col = [col for col in df.columns if col.startswith("DEL1 ")]  # Includes month-year
    worst_del_col = [col for col in df.columns if col.startswith("Worst_Del ")]

    if key_column in df.columns and del1_col and worst_del_col:
        selected_cols = [key_column] + del1_col + worst_del_col
        dfs_lst.append(df[selected_cols])  # Select only relevant columns

# Merge all DataFrames using ILS_ACCT_NUM as the primary key
merged_df = dfs_lst[0]
for df in dfs_lst[1:]:
    merged_df = pd.merge(merged_df, df, on=key_column, how="outer")

# Save the merged result
merged_df.to_csv("merged_car_1.csv", index=False)

# Display the first 100 rows
print(merged_df.head(100))

       ILS_ACCT_NUM  DEL1 JAN 2015  Worst_Del JAN 2015  DEL1 FEB 2015  \
0   200001000022100            1.0                 1.0            1.0   
1   200001000022115            1.0                 1.0            1.0   
2   200001000022121            1.0                 1.0            1.0   
3   200001000022148            1.0                 1.0            1.0   
4   200001000022154            1.0                 1.0            1.0   
..              ...            ...                 ...            ...   
95  200001000384245            0.0                 0.0            0.0   
96  200001000385469            0.0                 0.0            0.0   
97  200001000400166            2.0                 1.0            2.0   
98  200001000402343            0.0                 0.0            0.0   
99  200001000404016            0.0                 0.0            0.0   

    Worst_Del FEB 2015  DEL1 MAR 2015  Worst_Del MAR 2015  DEL1 APR 2015  \
0                  1.0            1.0          

In [2]:
folder_path = "C:/Users/Neil/OneDrive/Documents/Capstone Files/CAR5_8"
password = "CRQAD_amf_2025!"

# List all .xlsb files in the folder
xlsb_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsb")]

dfs1 = {}  # Dictionary to store DataFrames

for file in xlsb_files:
    file_path = os.path.join(folder_path, file)

    decrypted = io.BytesIO()
    with open(file_path, "rb") as f:
        office_file = msoffcrypto.OfficeFile(f)
        office_file.load_key(password=password)
        office_file.decrypt(decrypted)

    # Read decrypted .xlsb file
    df = []
    with open_workbook(decrypted) as wb:
        with wb.get_sheet(1) as sheet:  # Read first sheet
            for row in sheet.rows():
                df.append([item.v for item in row])

    df = pd.DataFrame(df[1:], columns=df[0])  # Convert to pandas DataFrame
    dfs1[file] = df  # Store DataFrame with file name as key

    print(f"Loaded: {file}")

Loaded: 49 ILS CAR JAN 2019.xlsb
Loaded: 50 ILS CAR FEB 2019.xlsb
Loaded: 51 ILS CAR MAR 2019.xlsb
Loaded: 52 ILS CAR APR 2019.xlsb
Loaded: 53 ILS CAR MAY 2019.xlsb
Loaded: 54 ILS CAR JUN 2019.xlsb
Loaded: 55 ILS CAR JUL 2019.xlsb
Loaded: 56  ILS CAR AUG 2019.xlsb
Loaded: 57 ILS CAR SEP 2019.xlsb
Loaded: 58 ILS CAR OCT 2019.xlsb
Loaded: 59 ILS CAR NOV 2019.xlsb
Loaded: 60 ILS CAR DEC 2019.xlsb
Loaded: 61 ILS CAR JAN 2020.xlsb
Loaded: 62 ILS CAR FEB 2020.xlsb
Loaded: 63 ILS CAR MAR 2020.xlsb
Loaded: 64 ILS CAR APR 2020.xlsb
Loaded: 65 ILS CAR MAY 2020.xlsb
Loaded: 66 ILS CAR JUN 2020.xlsb
Loaded: 67 ILS CAR JUL 2020.xlsb
Loaded: 68 ILS CAR AUG 2020.xlsb
Loaded: 69 ILS CAR SEP 2020.xlsb
Loaded: 70 ILS CAR OCT 2020.xlsb
Loaded: 71 ILS CAR NOV 2020.xlsb
Loaded: 72 ILS CAR DEC 2020.xlsb
Loaded: 73 ILS CAR JAN 2021.xlsb
Loaded: 74 ILS CAR FEB 2021.xlsb
Loaded: 75 ILS CAR MAR 2021.xlsb
Loaded: 76 ILS CAR APR 2021.xlsb
Loaded: 77 ILS CAR MAY 2021.xlsb
Loaded: 78 ILS CAR JUN 2021.xlsb
Loaded: 7

In [3]:
del_columns = ["DEL7", "DEL6", "DEL5", "DEL4", "DEL3", "DEL2", "DEL1"]

dfs_worst_delinquency1 = {}

for file, df in dfs1.items():
    existing_del_cols = [col for col in del_columns if col in df.columns]

    # Compute worst delinquency level correctly
    worst_del = df[existing_del_cols].ne(0).values.argmax(axis=1)

    # Fix issue: If all values are zero, set worst_del to 0
    all_zero_mask = df[existing_del_cols].sum(axis=1) == 0  # Identify all-zero rows
    df["Worst_Del"] = 7 - worst_del
    df.loc[all_zero_mask, "Worst_Del"] = 0  # Assign 0 to rows with no delinquency

    dfs_worst_delinquency1[file] = df

In [5]:
file_name = '49 ILS CAR JAN 2019.xlsb'  # Replace with the actual file name
dfs_worst_delinquency1[file_name][["DEL1", "DEL2", "DEL3", "DEL4", "DEL5", "DEL6", "DEL7", "Worst_Del"]].head(30)

,DEL1,DEL2,DEL3,DEL4,DEL5,DEL6,DEL7,Worst_Del
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,14.0,13.0,10.0,0.0,0.0,0.0,0.0,3
7,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [6]:
columns_to_rename = ["Worst_Del", "DEL1"]  # Include Worst_Del and DEL1

dfs_renamed1 = {}

for file, df in dfs_worst_delinquency1.items():
    # Extract the month and year (always at the end before .xlsb)
    filename = file.replace(".xlsb", "")  # Remove extension
    parts = filename.split()  # Split by spaces
    month_year = " ".join(parts[-2:])  # Get last two parts as "JAN 2015"

    # Rename columns
    rename_dict = {col: f"{col} {month_year}" for col in columns_to_rename if col in df.columns}
    
    df = df.rename(columns=rename_dict)
    
    # Store modified DataFrame
    dfs_renamed1[file] = df

# Example: Check a renamed DataFrame
print(dfs_renamed1["49 ILS CAR JAN 2019.xlsb"].head())

  APPRAISAL_DATE  APPRAISAL_RATE  APPRAISAL_VALUE   BOOK_DATE  BOOK_VALUE  \
0           None             0.0              0.0  03/21/2002         0.0   
1           None             0.0              0.0  03/21/2002         0.0   
2           None             0.0              0.0  10/25/2002         0.0   
3           None             0.0              0.0  02/19/2003         0.0   
4           None             0.0              0.0  05/15/2003         0.0   

                 CAR_DESCRIPTION  CAR_MANUFACTURER CAR_MODEL  CLOSE_DATE  \
0         2002 CHEVROLET VENTURE              10.0    2002.0  06/21/2004   
1          2002 TOYOTA ALTIS 1.6               9.0    2002.0  06/21/2005   
2  2002 ISUZU HILANDER WAGON XUV              12.0    2002.0  09/15/2006   
3          2003 ISUSU TROOPER AT              12.0    2003.0  01/10/2006   
4         2003 TOYOTA COROLLA MT               9.0    2003.0  03/20/2007   

   COLLATERAL_CONDITION  ...  NTC_AMT    RM_NO CLNT_TYPE  \
0                   

In [8]:
print(dfs_renamed1["49 ILS CAR JAN 2019.xlsb"].columns)

Index(['APPRAISAL_DATE', 'APPRAISAL_RATE', 'APPRAISAL_VALUE', 'BOOK_DATE',
       'BOOK_VALUE', 'CAR_DESCRIPTION', 'CAR_MANUFACTURER', 'CAR_MODEL',
       'CLOSE_DATE', 'COLLATERAL_CONDITION', 'COLLATERAL_TYPE',
       'CREDIT_RATING', 'PDO_MAT_DATE', 'DEL1 JAN 2019', 'DEL2', 'DEL3',
       'DEL4', 'DEL5', 'DEL6', 'DEL7', 'DELQ_AGE', 'DELQ_AGE_ACTUAL',
       'DOWNPAYMENT', 'EXPIRED', 'FACE_AMT', 'FINANCED_AMT', 'GL_STATUS',
       'GL_STATUS_DESCRIPTION', 'GROSS_MONTHLY_AMORTISATION', 'ILS_ACCT_NUM',
       'LAND_VALUE', 'LAST_PAID_AMT', 'LAST_PAID_DATE', 'LOAN_DUE', 'MATURED',
       'MATURITY', 'MATURITY_DUE_CHANGE', 'NMA', 'ORIG_BOOK_DATE',
       'ORIGINAL_RATE', 'OUTSTANDING_BAL', 'PREPAYMENT', 'PRODUCT_TYPE',
       'PROPERTY_TYPE', 'LOAN_TERM', 'WORST_DELINQUENCY', 'COLLATERAL_REFNUM',
       'PURPOSE_CODE', 'PURPOSE_DESC', 'REMEDIAL_STATUS_CODE',
       'PAST_DUE_STATUS', 'DEFAULT_TAGGING', 'EXCHANGE_RATE', 'SECURITY_TYPE',
       'RE_FLOOR_AREA', 'AVAILMENT', 'CAR_TYPE', 'LOA

In [9]:
key_column = "ILS_ACCT_NUM"

# List to store DataFrames with selected columns
dfs_lst1 = []

for file, df in dfs_renamed1.items():
    # Identify columns that start with 'DEL1' and 'Worst_Del'
    del1_col = [col for col in df.columns if col.startswith("DEL1 ")]  # Includes month-year
    worst_del_col = [col for col in df.columns if col.startswith("Worst_Del ")]

    if key_column in df.columns and del1_col and worst_del_col:
        selected_cols = [key_column] + del1_col + worst_del_col
        dfs_lst1.append(df[selected_cols])  # Select only relevant columns

# Merge all DataFrames using ILS_ACCT_NUM as the primary key
merged_df1 = dfs_lst1[0]
for df in dfs_lst1[1:]:
    merged_df1 = pd.merge(merged_df1, df, on=key_column, how="outer")

# Save the merged result
merged_df1.to_csv("merged_car_2.csv", index=False)

# Display the first 100 rows
print(merged_df1.head(100))

       ILS_ACCT_NUM  DEL1 JAN 2019  Worst_Del JAN 2019  DEL1 FEB 2019  \
0   200001000022100            1.0                 1.0            1.0   
1   200001000022115            1.0                 1.0            1.0   
2   200001000022121            1.0                 1.0            1.0   
3   200001000022148            1.0                 1.0            1.0   
4   200001000022154            1.0                 1.0            1.0   
..              ...            ...                 ...            ...   
95  200001000384245            0.0                 0.0            0.0   
96  200001000385469            0.0                 0.0            0.0   
97  200001000400166            2.0                 1.0            2.0   
98  200001000402343            0.0                 0.0            0.0   
99  200001000404016            0.0                 0.0            0.0   

    Worst_Del FEB 2019  DEL1 MAR 2019  Worst_Del MAR 2019  DEL1 APR 2019  \
0                  1.0            1.0          

In [8]:
folder_path = "C:/Users/Neil/OneDrive/Documents/Capstone Files/CAR9_10"
password = "CRQAD_amf_2025!"

# List all .xlsb files in the folder
xlsb_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsb")]

dfs2 = {}  # Dictionary to store DataFrames

for file in xlsb_files:
    file_path = os.path.join(folder_path, file)

    decrypted = io.BytesIO()
    with open(file_path, "rb") as f:
        office_file = msoffcrypto.OfficeFile(f)
        office_file.load_key(password=password)
        office_file.decrypt(decrypted)

    # Read decrypted .xlsb file
    df = []
    with open_workbook(decrypted) as wb:
        with wb.get_sheet(1) as sheet:  # Read first sheet
            for row in sheet.rows():
                df.append([item.v for item in row])

    df = pd.DataFrame(df[1:], columns=df[0])  # Convert to pandas DataFrame
    dfs2[file] = df  # Store DataFrame with file name as key

    print(f"Loaded: {file}")

Loaded: 97 ILS CAR JAN 2023.xlsb
Loaded: 98 ILS CAR FEB 2023.xlsb
Loaded: 99 ILS CAR MAR 2023.xlsb
Loaded: 9901 ILS CAR APR 2023.xlsb
Loaded: 9902 ILS CAR MAY 2023.xlsb
Loaded: 9903 ILS CAR JUN 2023.xlsb
Loaded: 9904 ILS CAR JUL 2023.xlsb
Loaded: 9905 ILS CAR AUG 2023.xlsb
Loaded: 9906 ILS CAR SEP 2023.xlsb
Loaded: 9907 ILS CAR OCT 2023.xlsb
Loaded: 9908 ILS CAR NOV 2023.xlsb
Loaded: 9909 ILS CAR DEC 2023.xlsb
Loaded: 9910 ILS CAR JAN 2024.xlsb
Loaded: 9911 ILS CAR FEB 2024.xlsb
Loaded: 9912 ILS CAR MAR 2024.xlsb
Loaded: 9913 ILS CAR APR 2024.xlsb
Loaded: 9914 ILS CAR MAY 2024.xlsb
Loaded: 9915 ILS CAR JUN 2024.xlsb
Loaded: 9916 ILS CAR JUL 2024.xlsb
Loaded: 9917 ILS CAR AUG 2024.xlsb
Loaded: 9918 ILS CAR SEP 2024.xlsb
Loaded: 9919 ILS CAR OCT 2024.xlsb
Loaded: 9920 ILS CAR NOV 2024.xlsb
Loaded: 9921 ILS CAR DEC 2024.xlsb


In [9]:
del_columns = ["DEL7", "DEL6", "DEL5", "DEL4", "DEL3", "DEL2", "DEL1"]

dfs_worst_delinquency2 = {}

for file, df in dfs2.items():
    existing_del_cols = [col for col in del_columns if col in df.columns]

    # Compute worst delinquency level correctly
    worst_del = df[existing_del_cols].ne(0).values.argmax(axis=1)

    # Fix issue: If all values are zero, set worst_del to 0
    all_zero_mask = df[existing_del_cols].sum(axis=1) == 0  # Identify all-zero rows
    df["Worst_Del"] = 7 - worst_del
    df.loc[all_zero_mask, "Worst_Del"] = 0  # Assign 0 to rows with no delinquency

    dfs_worst_delinquency2[file] = df

In [10]:
file_name = '97 ILS CAR JAN 2023.xlsb'  # Replace with the actual file name
dfs_worst_delinquency2[file_name][["DEL1", "DEL2", "DEL3", "DEL4", "DEL5", "DEL6", "DEL7", "Worst_Del"]].head(30)

,DEL1,DEL2,DEL3,DEL4,DEL5,DEL6,DEL7,Worst_Del
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,14.0,13.0,10.0,0.0,0.0,0.0,0.0,3
7,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [11]:
columns_to_rename = ["Worst_Del", "DEL1"]  # Include Worst_Del and DEL1

dfs_renamed2 = {}

for file, df in dfs_worst_delinquency2.items():
    # Extract the month and year (always at the end before .xlsb)
    filename = file.replace(".xlsb", "")  # Remove extension
    parts = filename.split()  # Split by spaces
    month_year = " ".join(parts[-2:])  # Get last two parts as "JAN 2015"

    # Rename columns
    rename_dict = {col: f"{col} {month_year}" for col in columns_to_rename if col in df.columns}
    
    df = df.rename(columns=rename_dict)
    
    # Store modified DataFrame
    dfs_renamed2[file] = df

# Example: Check a renamed DataFrame
print(dfs_renamed2["97 ILS CAR JAN 2023.xlsb"].head())

  APPRAISAL_DATE  APPRAISAL_RATE  APPRAISAL_VALUE   BOOK_DATE  BOOK_VALUE  \
0           None             0.0              0.0  03/21/2002         0.0   
1           None             0.0              0.0  03/21/2002         0.0   
2           None             0.0              0.0  10/25/2002         0.0   
3           None             0.0              0.0  02/19/2003         0.0   
4           None             0.0              0.0  05/15/2003         0.0   

                 CAR_DESCRIPTION  CAR_MANUFACTURER CAR_MODEL  CLOSE_DATE  \
0         2002 CHEVROLET VENTURE              10.0    2002.0  06/21/2004   
1          2002 TOYOTA ALTIS 1.6               9.0    2002.0  06/21/2005   
2  2002 ISUZU HILANDER WAGON XUV              12.0    2002.0  09/15/2006   
3          2003 ISUSU TROOPER AT              12.0    2003.0  01/10/2006   
4         2003 TOYOTA COROLLA MT               9.0    2003.0  03/20/2007   

   COLLATERAL_CONDITION  ...  NTC_AMT    RM_NO CLNT_TYPE  \
0                   

In [12]:
print(dfs_renamed2["97 ILS CAR JAN 2023.xlsb"].columns)

Index(['APPRAISAL_DATE', 'APPRAISAL_RATE', 'APPRAISAL_VALUE', 'BOOK_DATE',
       'BOOK_VALUE', 'CAR_DESCRIPTION', 'CAR_MANUFACTURER', 'CAR_MODEL',
       'CLOSE_DATE', 'COLLATERAL_CONDITION', 'COLLATERAL_TYPE',
       'CREDIT_RATING', 'PDO_MAT_DATE', 'DEL1 JAN 2023', 'DEL2', 'DEL3',
       'DEL4', 'DEL5', 'DEL6', 'DEL7', 'DELQ_AGE', 'DELQ_AGE_ACTUAL',
       'DOWNPAYMENT', 'EXPIRED', 'FACE_AMT', 'FINANCED_AMT', 'GL_STATUS',
       'GL_STATUS_DESCRIPTION', 'GROSS_MONTHLY_AMORTISATION', 'ILS_ACCT_NUM',
       'LAND_VALUE', 'LAST_PAID_AMT', 'LAST_PAID_DATE', 'LOAN_DUE', 'MATURED',
       'MATURITY', 'MATURITY_DUE_CHANGE', 'NMA', 'ORIG_BOOK_DATE',
       'ORIGINAL_RATE', 'OUTSTANDING_BAL', 'PREPAYMENT', 'PRODUCT_TYPE',
       'PROPERTY_TYPE', 'LOAN_TERM', 'WORST_DELINQUENCY', 'COLLATERAL_REFNUM',
       'PURPOSE_CODE', 'PURPOSE_DESC', 'REMEDIAL_STATUS_CODE',
       'PAST_DUE_STATUS', 'DEFAULT_TAGGING', 'EXCHANGE_RATE', 'SECURITY_TYPE',
       'RE_FLOOR_AREA', 'AVAILMENT', 'CAR_TYPE', 'LOA

In [13]:
key_column = "ILS_ACCT_NUM"

# List to store DataFrames with selected columns
dfs_lst2 = []

for file, df in dfs_renamed2.items():
    # Identify columns that start with 'DEL1' and 'Worst_Del'
    del1_col = [col for col in df.columns if col.startswith("DEL1 ")]  # Includes month-year
    worst_del_col = [col for col in df.columns if col.startswith("Worst_Del ")]

    if key_column in df.columns and del1_col and worst_del_col:
        selected_cols = [key_column] + del1_col + worst_del_col
        dfs_lst2.append(df[selected_cols])  # Select only relevant columns

# Merge all DataFrames using ILS_ACCT_NUM as the primary key
merged_df2 = dfs_lst2[0]
for df in dfs_lst2[1:]:
    merged_df2 = pd.merge(merged_df2, df, on=key_column, how="outer")

# Save the merged result
merged_df2.to_csv("merged_car_3.csv", index=False)

# Display the first 100 rows
print(merged_df2.head(100))

       ILS_ACCT_NUM  DEL1 JAN 2023  Worst_Del JAN 2023  DEL1 FEB 2023  \
0   200001000022100            1.0                 1.0            1.0   
1   200001000022115            1.0                 1.0            1.0   
2   200001000022121            1.0                 1.0            1.0   
3   200001000022148            1.0                 1.0            1.0   
4   200001000022154            1.0                 1.0            1.0   
..              ...            ...                 ...            ...   
95  200001000384245            0.0                 0.0            0.0   
96  200001000385469            0.0                 0.0            0.0   
97  200001000400166            2.0                 1.0            2.0   
98  200001000402343            0.0                 0.0            0.0   
99  200001000404016            0.0                 0.0            0.0   

    Worst_Del FEB 2023  DEL1 MAR 2023  Worst_Del MAR 2023  DEL1 APR 2023  \
0                  1.0            1.0          